# Lab 5. Packages analysis - IDS
Sergio Marchena - 16387


2022

## parte 1 - análisis de paquetes 

1. Capture 100 paquetes con la herramienta scapy, imprima el tipo de datos, la longitud y el contenido del pcap 